In [2]:
# Notebook setup: ensure we run from repo root (so relative paths like configs/ and runs/ work)
import os
import sys
from pathlib import Path

def _find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p] + list(p.parents):
        if (cand / "pyproject.toml").exists() and (cand / "src").exists():
            return cand
    # Fallback: if executed from notebooks/, go one level up
    if p.name.lower() == "notebooks" and (p.parent / "src").exists():
        return p.parent
    return p

REPO_ROOT = _find_repo_root()
os.chdir(REPO_ROOT)

src_path = REPO_ROOT / "src"
if src_path.exists() and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)

#Tip: for faster iteration, set `baselines.login_attempt.preset: fast` in your config. This clamps expensive hyperparams (e.g., RF trees) while keeping the same data/splits/metrics.


Repo root: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab


# BaselineLab — login_attempt


This notebook trains BaselineLab models (LogReg + HGB) on the FeatureLab output and prints the generated report path.


In [3]:
from pathlib import Path
from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.models import run_login_baselines_for_run

cfg = load_config(Path("configs/skynet_smoke.yaml"))
run_id = cfg.run.run_id #or "RUN_XXX_0005"  # set to your run id

run_login_baselines_for_run(cfg, run_id=run_id, force=True)

report = Path(cfg.paths.runs_dir) / run_id / "models" / "login_attempt" / "baselines" / "report.md"
print(report)
print(report.read_text(encoding="utf-8")[:1000])


runs\RUN_SAMPLE_SMOKE_0001\models\login_attempt\baselines\report.md
# Baseline Report — login_attempt

- run_id: `RUN_SAMPLE_SMOKE_0001`
- target_fpr: 0.01
- rows: train=8142, time_eval=1184, user_holdout=1476

## Summary table

| label | model | train PR-AUC | train thr | train FPR | train recall | time_eval PR-AUC | time_eval FPR | time_eval recall | user_holdout PR-AUC | user_holdout FPR | user_holdout recall |
|---|---|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| label_replicators | logreg | 0.0556 | 0.857271 | 0.0100 | 0.0287 | 0.0214 | 0.0181 | 0.0000 | 0.0255 | 0.0069 | 0.0000 |
| label_replicators | rf | 1.0000 | 0.0766667 | 0.0097 | 1.0000 | 0.0224 | 0.1002 | 0.0385 | 0.0257 | 0.0763 | 0.0571 |
| label_the_mule | logreg | 0.0441 | 0.942879 | 0.0099 | 0.0206 | 0.0146 | 0.0137 | 0.0000 | 0.0148 | 0.0130 | 0.0000 |
| label_the_mule | rf | 1.0000 | 0.0533333 | 0.0089 | 1.0000 | 0.0195 | 0.0883 | 0.0556 | 0.0164 | 0.0446 | 0.1053 |
| label_the_chameleon | logreg | 0.0478 | 

# Notebook visibility helpers

Use these helpers to quickly locate artifacts and tail logs (so long runs feel more transparent).


In [ ]:
from pathlib import Path
from inkswarm_detectlab.ui.notebook_tools import find_run_dir, print_run_tree, tail_text

#RUN_ID = "RR2_MVP_ZIP_A_0002"  # <-- change me
RUN_ID = run_id
ROOT = Path("..").resolve() if Path.cwd().name == "notebooks" else Path(".").resolve()
run_dir = find_run_dir(ROOT, RUN_ID)
print_run_tree(run_dir)

print("\n--- baselines.log (tail) ---\n")
print(tail_text(run_dir / "share/logs/baselines.log", n_lines=200))
